# python to gephi using LLM

In [396]:
## connect to gephi server
from gephistreamer import graph
from gephistreamer import streamer

In [397]:
# create a stream 
stream = streamer.Streamer(streamer.GephiWS(hostname="localhost", port=8080, workspace="workspace1"))


# extract data from Web URL

In [398]:
# extract data from web url
import os 

from langchain_core.documents import Document
from helpers.clean_data import clean_text
from langchain_community.document_loaders import WebBaseLoader

In [411]:
url_list=[]


urls = [
    'https://en.wikipedia.org/wiki/Dana_White',
    'https://en.wikipedia.org/wiki/Elon_Musk',
    'https://en.wikipedia.org/wiki/Mark_Zuckerberg',
    'https://en.wikipedia.org/wiki/Bill_Gates',
    'https://en.wikipedia.org/wiki/Jeff_Bezos',
    'https://en.wikipedia.org/wiki/Steve_Jobs',
    'https://en.wikipedia.org/wiki/Sam_Altman',
    'https://en.wikipedia.org/wiki/Mukesh_Ambani',
    'https://en.wikipedia.org/wiki/Jensen_Huang',
    'https://en.wikipedia.org/wiki/Satoshi_Nakamoto',
    'https://en.wikipedia.org/wiki/Donald_Trump',
    'https://en.wikipedia.org/wiki/Larry_Ellison',
    'https://en.wikipedia.org/wiki/Larry_Page',
    'https://en.wikipedia.org/wiki/Mark_Cuban',
    'https://en.wikipedia.org/wiki/Ratan_Tata',
    'https://en.wikipedia.org/wiki/Brian_Chesky',
    'https://en.wikipedia.org/wiki/Jack_Dorsey',
    'https://en.wikipedia.org/wiki/Satya_Nadella'
]


for url in urls:
    url_list.append(url)

    loader=WebBaseLoader([url])
    data=loader.load().pop().page_content
data

'\n\n\n\nSatya Nadella - Wikipedia\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nJump to content\n\n\n\n\n\n\n\nMain menu\n\n\n\n\n\nMain menu\nmove to sidebar\nhide\n\n\n\n\t\tNavigation\n\t\n\n\nMain pageContentsCurrent eventsRandom articleAbout WikipediaContact us\n\n\n\n\n\n\t\tContribute\n\t\n\n\nHelpLearn to editCommunity portalRecent changesUpload file\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nAppearance\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nDonate\n\nCreate account\n\nLog in\n\n\n\n\n\n\n\n\nPersonal tools\n\n\n\n\n\nDonate Create account Log in\n\n\n\n\n\n\t\tPages for logged out editors learn more\n\n\n\nContributionsTalk\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nContents\nmove to sidebar\nhide\n\n\n\n\n(Top)\n\n\n\n\n\n1\nEarly life\n\n\n\n\n\n\n\n\n2\nCareer\n\n\n\n\nToggle Career subsection\n\n\n\n\n\n2.1\nSun Microsystems\n\n\n\n\n\n\n\n\n2.2\nMicrosoft\n\n\n\

In [412]:
from langchain.text_splitter import RecursiveCharacterTextSplitter


loader=WebBaseLoader([url])
data=loader.load().pop().page_content
data=clean_text(data)
documents=[Document(page_content=data)]
# print(documents)
splitter=RecursiveCharacterTextSplitter(chunk_size=3000,chunk_overlap=100)
smaller_doc=splitter.split_documents(documents)
len(smaller_doc)


10

In [401]:
import os 
from groq import Groq

In [402]:
client=Groq( api_key=os.getenv('GROQ_API_KEY')
)

# try nodes and relationships


In [403]:
system=""" You are a network graph maker tasked with extracting terms and their relations from a given context. Your job is to analyze the provided context chunk (delimited by ```) and extract the ontology of terms mentioned in the context. These terms should represent the key concepts based on the context.

**Guidelines for Extraction:**

1. **Identify Key Terms**:
   - While analyzing the context, identify key terms from each sentence or paragraph.
   - Key terms may include:
     - Objects
     - Entities
     - Locations
     - Organizations
     - Persons
     - Conditions
     - Acronyms
     - Documents
     - Services
     - Concepts, etc.
   - Ensure the terms are as **atomistic** and specific as possible.
   - Ensure to give response only in JSON format and no other texts or anything to indicate the format

2. **Identify Relationships**:
   - Terms mentioned in the same sentence or paragraph are typically related.
   - Each term can have one-on-one relationships with other terms.
   - Determine how terms are related to one another.

3. **Define Relationships**:
   - Clearly identify the nature of the relationship between related terms.
   - Use concise and descriptive language to explain the connection.

**Output Format:**
- Provide your output as a list of JSON objects, with a maximum of 5 term pairs.
- Each object should include:

  - `"node_1"`: A concept extracted from the context.
  - `"node_2"`: A related concept extracted from the context.
  - `"edge"`: A concise sentence describing the relationship between `node_1` and `node_2`.

**Example Output:**
[
   {   
       "node_1": "Climate change",
       "node_2": "Greenhouse gases",
       "edge": "Greenhouse gases contribute significantly to climate change."
   },
   {
       
       "node_1": "Tesla, Inc.",
       "node_2": "Elon Musk",
       "edge": "Elon Musk is the founder and CEO of Tesla, Inc."
   }
]"""

In [410]:
def get_nodes_and_edges(smaller_doc):
    results=[]
    for doc in smaller_doc[:100]:
        chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": system
            },
            {
                "role": "user",
                "content": doc.page_content,
            }
        ],
        model='llama-3.1-8b-instant',
        )
        results.append(chat_completion.choices[0].message.content)
    return results

In [405]:
print(len(results))


10


In [406]:
from gephistreamer import graph
from gephistreamer import streamer
import json 

stream = streamer.Streamer(streamer.GephiWS(hostname="localhost", port=8080, workspace="workspace1"))


In [407]:
for res in results:
    try: 
        json_object=json.loads(res)
        print(res)
        for data in json_object:
            node_a = graph.Node(data['node_1'],custom_property=1)
            node_b = graph.Node(data['node_2'],custom_property=2)
            stream.add_node(node_a,node_b)
            edge_ab = graph.Edge(node_a,node_b,custom_property=data['edge'])
            stream.add_edge(edge_ab)

    except Exception as e:
        print('buggy JSON object', e)
    
    

[
  {
    "node_1": "Satya Nadella",
    "node_2": "Microsoft",
    "edge": "Satya Nadella is the chairman and chief executive officer of Microsoft."
  },
  {
    "node_1": "Microsoft",
    "node_2": "Steve Ballmer",
    "edge": "Satya Nadella succeeded Steve Ballmer as CEO of Microsoft."
  },
  {
    "node_1": "Nadella",
    "node_2": "Hyderabad",
    "edge": "Satya Nadella was born in Hyderabad, India into a Telugu Hindu family."
  },
  {
    "node_1": "Satya Nadella",
    "node_2": "Anupama Nadella",
    "edge": "Satya Nadella is married to Anupama Nadella."
  },
  {
    "node_1": "Satya Nadella",
    "node_2": "University of Chicago",
    "edge": "Satya Nadella received an MBA from the University of Chicago Booth."
  }
]
[
   {
      "node_1": "Satya Nadella",
      "node_2": "Microsoft",
      "edge": "Satya Nadella worked at Microsoft as a CEO from 2014"
   },
   {
      "node_1": "Cloud computing",
      "node_2": "Microsoft",
      "edge": "Nadella led the company's move to clo